image croping and resteration

In [17]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Model, load_model

# single image code
# img = cv2.imread(r'C:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\mask\Mask_1.tif', cv2.IMREAD_UNCHANGED)
# print(f'dtype: {img.dtype}, shape: {img.shape}, min: {np.min(img)}, max: {np.max(img)}')
# img

# batch processing
folder = r"c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\Test"
image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]
print(f'Total number of images: {len(image_files)}')

# output of fragments 
output_folder = r'c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\258x258 Test'

## Define the dimensions of the sub-images
section_height = 256
section_width = 256
overlap = 51  # 50% overlap

Total number of images: 5


In [7]:
# single image
# Calculate the number of sections along height and width of a gray scale image
height, width = img.shape
sections_vertical = (height - section_height) // overlap + 1
sections_horizontal = (width - section_width) // overlap + 1

# # List to store all the 500x500 sections
# sections = []

# # Loop through the image to split it into 500x500 sections with overlap
# for i in range(sections_vertical):
#     for j in range(sections_horizontal):
#         y_start = i * overlap
#         x_start = j * overlap
#         y_end = y_start + section_height
#         x_end = x_start + section_width
        
#         section = img[y_start:y_end, x_start:x_end]
#         sections.append(section)

# # Now 'sections' contains all the 500x500 overlapping sections
# print(f'Total sections: {len(sections)}')

# # Optionally, show one of the sections (for verification)
# # WARNING code will open a new window for the image to display in, the code will continue running until the window is closed.
# # cv2.imshow('Section', sections[0])
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# # confirmed code works as intended

In [8]:
os.makedirs(output_folder, exist_ok=True)

print(f"Creating {sections_vertical} vertical sections and {sections_horizontal} horizontal sections.")

section_counter = 1
for i in range(sections_vertical):
    for j in range(sections_horizontal):
        y_start = i * overlap
        x_start = j * overlap
        y_end = y_start + section_height
        x_end = x_start + section_width
        
        # Ensure the last section doesn't go out of bounds
        y_end = min(y_end, height)
        x_end = min(x_end, width)
        
        # Slice the image and save the section
        section = img[y_start:y_end, x_start:x_end]
        
        # Construct the filename for saving
        file_name = os.path.join(output_folder, f"section_{section_counter:03d}.tiff")  # Save as TIFF
        # For PNG use: file_name = os.path.join(output_folder, f"section_{section_counter:03d}.png")
        
        # Save the section as a TIFF or PNG
        cv2.imwrite(file_name, section)
        
        # Increment the section counter
        section_counter += 1

print(f"Saved {section_counter - 1} sections to the '{output_folder}' folder.")

Creating 13 vertical sections and 12 horizontal sections.
Saved 156 sections to the 'c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\258x258 Test' folder.


In [ ]:
# batch process
os.makedirs(output_folder, exist_ok=True)

# Initialize a section counter
section_counter = 1

# Loop through each image in the folder
for image_file in image_files:
    img_path = os.path.join(folder, image_file)
    
    # Read the image in grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Use cv2.IMREAD_GRAYSCALE for grayscale images
    
    if img is None:
        print(f"Failed to load image: {image_file}")
        continue  # Skip this image if it failed to load
    
    height, width = img.shape

    # Calculate the number of sections along height and width of the image
    sections_vertical = (height - section_height) // overlap + 1
    sections_horizontal = (width - section_width) // overlap + 1

    # Print information about the current image
    print(f"Processing image: {image_file}")
    print(f"Creating {sections_vertical} vertical sections and {sections_horizontal} horizontal sections.")

    # Loop through the image to split it into sections with overlap
    for i in range(sections_vertical):
        for j in range(sections_horizontal):
            y_start = i * overlap
            x_start = j * overlap
            y_end = y_start + section_height
            x_end = x_start + section_width
            
            # Ensure the last section doesn't go out of bounds
            y_end = min(y_end, height)
            x_end = min(x_end, width)
            
            # Slice the image and save the section
            section = img[y_start:y_end, x_start:x_end]
            
            # Construct the filename for saving
            file_name = os.path.join(output_folder, f"section_{section_counter:03d}.tiff")  # Save as TIFF
            # For PNG use: file_name = os.path.join(output_folder, f"section_{section_counter:03d}.png")
            
            # Save the section as a TIFF or PNG
            cv2.imwrite(file_name, section)
            
            # Increment the section counter
            section_counter += 1

    print(f"Finished processing {image_file}")

# Final output: Total sections saved
print(f"Saved {section_counter - 1} sections to the '{output_folder}' folder.")

Processing image: Gel_26.tif
Creating 13 vertical sections and 12 horizontal sections.
Finished processing Gel_26.tif
Processing image: Gel_27.tif
Creating 12 vertical sections and 11 horizontal sections.
Finished processing Gel_27.tif
Processing image: Gel_28.tif
Creating 12 vertical sections and 11 horizontal sections.
Finished processing Gel_28.tif
Processing image: Gel_29.tif
Creating 12 vertical sections and 11 horizontal sections.
Finished processing Gel_29.tif
Processing image: Gel_30.tif
Creating 12 vertical sections and 11 horizontal sections.
Finished processing Gel_30.tif
Saved 684 sections to the 'c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\258x258 Test' folder.


In [14]:
# restoration code

# Create a blank canvas to hold the reconstructed image
reconstructed_img = np.zeros((1824, 1614), dtype=np.uint8)

# Loop through 'sections' list from previous code
section_counter = 0  # first section

# Loop through file sections to reconstruct the image
# section_counter = 1 # first file in folder
for i in range(sections_vertical):
    for j in range(sections_horizontal):
        y_start = i * overlap
        x_start = j * overlap
        y_end = y_start + section_height
        x_end = x_start + section_width

        # for sections code
        section = sections[section_counter]

        # for folder code 
        # file_name = os.path.join('sections', f"section_{section_counter:03d}.tiff")  # TIFF file format
        # section = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)

        # Ensure the section fits inside the boundaries
        y_end = min(y_end, reconstructed_img.shape[0])
        x_end = min(x_end, reconstructed_img.shape[1])

        # Add the section to the reconstructed image, averaging if there is overlap
        reconstructed_img[y_start:y_end, x_start:x_end] = section[:y_end-y_start, :x_end-x_start]
        
        section_counter += 1

# Show the reconstructed image
# WARNING reconstructions of images can max out the computers ability to project the image if it is larger than the computers display. 
# Reconstruction will work better if saved as a new file
# cv2.imshow('Reconstructed Image', reconstructed_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Optionally, save the reconstructed image
cv2.imwrite('reconstructed_image.tiff', reconstructed_img)

True

In [15]:
# Function to apply binning and reassign intensity values
def bin_and_reassign_intensities(image, bins, new_values):
    """
    Bins the pixel intensities and assigns new values to each bin.
    :param image: Input image (numpy array).
    :param bins: List of bin edges.
    :param new_values: List of new intensity values corresponding to each bin.
    :return: Image with reassigned intensities.
    """
    # Flatten the image to work with a 1D array of pixel intensities
    flat_image = image.flatten()

    # Assign the new intensity values based on the bin
    binned_values = np.digitize(flat_image, bins) - 1  # -1 because bins start from 1

    # Ensure that bin indices are within range
    binned_values = np.clip(binned_values, 0, len(new_values) - 1)

    # Assign new intensity values to pixels based on their bin index
    new_intensities = np.array([new_values[i] for i in binned_values])

    # Reshape the 1D array back to the original image shape
    new_image = new_intensities.reshape(image.shape)

    return new_image

In [10]:
model = load_model(r'C:\Users\jackz\Documents\Homework\Data Sciance\STT 811\Project\Gel_Train_res_3_limit.keras')
model.summary()

Model: "RESUNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 256, 256,  │        640 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_60[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_49       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 256, 256,  │     36,928 │ activation_49[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 256, 256,  │        128 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 256, 256,  │          0 │ conv2d_61[0][0],  │
│                     │ 64)               │            │ conv2d_62[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ add_7[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_50       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 128, 128,  │     73,856 │ activation_50[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_63[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_51       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 128, 128,  │    147,584 │ activation_51[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_65 (Conv2D)  │ (None, 128, 128,  │      8,320 │ add_7[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 128, 128,  │          0 │ conv2d_64[0][0],  │
│                     │ 128)              │            │ conv2d_65[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ add_8[0][0]       │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_52       │ (None, 128, 128,  │          0 │ batch_normalizat

 Total params: 24,665,541 (94.09 MB)

 Trainable params: 8,219,713 (31.36 MB)

 Non-trainable params: 6,400 (25.00 KB)

 Optimizer params: 16,439,428 (62.71 MB)

In [ ]:
# Folder paths
input_folder = r"c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\Test"
output_folder = r'c:\Users\jackz\Pictures\Gel Img Train data\Lab Gels\Test predict'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define image splitting parameters
section_height = 256
section_width = 256
overlap = 205  # 20% overlap

# Get list of images in the input folder
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]
print(f'Total number of images: {len(image_files)}')

# Process each image in the folder
for image_file in image_files:
    # Load the image
    img_path = os.path.join(input_folder, image_file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale, modify if needed
    height, width = img.shape
    reconstructed_img = np.zeros((height, width), dtype=np.uint8)
    # Calculate number of sections
    sections_vertical = (height - section_height) // overlap + 1
    sections_horizontal = (width - section_width) // overlap + 1
    print(f"Creating {sections_vertical} vertical sections and {sections_horizontal} horizontal sections for {image_file}")
    
    section_counter = 1
    for i in range(sections_vertical):
        for j in range(sections_horizontal):
            y_start = i * overlap
            x_start = j * overlap
            y_end = y_start + section_height
            x_end = x_start + section_width
            
            # Ensure the last section doesn't go out of bounds
            y_end = min(y_end, height)
            x_end = min(x_end, width)
            
            # Slice the image into sections
            section = img[y_start:y_end, x_start:x_end]

            # Prepare the batch of images for prediction (even if it’s just one section)
            batch_images = np.expand_dims(section, axis=-1)
            batch_images = np.expand_dims(section, axis=0)  # Add batch dimension for model input

            # Run prediction for the section (model should output a prediction per section)
            predictions = model.predict(batch_images)

            # You may need to post-process predictions, depending on your model (e.g., if it's segmentation or classification)
            mean = predictions.mean()
            bins = [0, mean/10, mean/8, mean/6, mean/4, mean/2, mean, mean*2, mean*4, mean*6, mean*8, mean*10, 255]

            # Define the new intensity values for each bin (e.g., [10, 60, 120, 180, 240])
            # These values will be assigned to the pixels in their respective bins
            new_values = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

            # Apply binning and reassign intensities
            new_image_bin = bin_and_reassign_intensities(predictions, bins, new_values)
            # Save the predicted section (if the model's output is an image-like array)
            predicted_section = new_image_bin[0]  # Assuming model returns 1 result per input
            
            predicted_section = np.squeeze(predicted_section)
            
            # Save the predicted section to the output folder
            section_filename = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_section_{section_counter:03d}.tiff")
            cv2.imwrite(section_filename, predicted_section)  # Save the predicted section

            # Optionally, update the reconstructed image by combining the predicted section (you could average if overlap exists)
            reconstructed_img[y_start:y_end, x_start:x_end] = predicted_section[:y_end - y_start, :x_end - x_start]

            # Increment the section counter
            section_counter += 1

    # Optionally, save the reconstructed image
    reconstructed_image_filename = os.path.join(output_folder, f"reconstructed_{os.path.splitext(image_file)[0]}.tiff")
    cv2.imwrite(reconstructed_image_filename, reconstructed_img)
    print(f"Reconstructed image saved: {reconstructed_image_filename}")




Total number of images: 5
Creating 8 vertical sections and 7 horizontal sections for Gel_26.tif
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


KeyboardInterrupt: 